In [1]:
import requests
import pandas as pd
import numpy as np

# Get the data

In [3]:
# fuction to get the data 


# Function to download a file from a URL and save it in the `data` folder
def download_file(url, local_filename):
    response = requests.get(url)
    if response.status_code == 200:
        with open(local_filename, 'wb') as file:
            file.write(response.content)
        print(f"Downloaded {local_filename}")
    else:
        print(f"Failed to download {local_filename}. Status code: {response.status_code}")

# URLs for the files
url_orders = 'https://github.com/Digital-IFCO/data-engineering-test/blob/main/resources/orders.csv'  # URL of orders.csv
url_invoicing = 'https://github.com/Digital-IFCO/data-engineering-test/blob/main/resources/invoicing_data.json'  # URL of invoicing_data.json

# Download the files into the `data` folder
download_file(url_orders, '../data/orders.csv')
download_file(url_invoicing, '../data/invoicing_data.json')

Downloaded ../data/orders.csv
Downloaded ../data/invoicing_data.json


In [8]:

# Cargar el archivo orders.csv
orders = pd.read_csv('../data/orders.csv',delimiter=';')

# Verificar las primeras filas para revisar la estructura de los datos
orders.head()

,order_id,date,company_id,company_name,crate_type,contact_data,salesowners
0,f47ac10b-58cc-4372-a567-0e02b2c3d479,29.01.22,1e2b47e6-499e-41c6-91d3-09d12dddfbbd,Fresh Fruits Co,Plastic,"[{ ""contact_name"":""Curtis"", ""contact_surname"":...","Leonard Cohen, Luke Skywalker, Ammy Winehouse"
1,f47ac10b-58cc-4372-a567-0e02b2c3d480,21.02.22,0f05a8f1-2bdf-4be7-8c82-4c9b58f04898,Veggies Inc,Wood,"[{ ""contact_name"":""Maria"", ""contact_surname"":""...","Luke Skywalker, David Goliat, Leon Leonov"
2,f47ac10b-58cc-4372-a567-0e02b2c3d481,03.04.22,1e2b47e6-499e-41c6-91d3-09d12dddfbbd,Fresh Fruits c.o,Metal,"[{ ""contact_name"":""Para"", ""contact_surname"":""C...",Luke Skywalker
3,f47ac10b-58cc-4372-a567-0e02b2c3d482,14.07.21,1c4b0b50-1d5d-463a-b56e-1a6fd3aeb7d6,Seafood Supplier,Plastic,NaN,"David Goliat, Leonard Cohen"
4,f47ac10b-58cc-4372-a567-0e02b2c3d483,23.10.22,34538e39-cd2e-4641-8d24-3c94146e6f16,Meat Packers Ltd,Plastic,NaN,"Chris Pratt, David Henderson, Marianov Merschi..."


# Distribution of Crate Type per Company


In [12]:
import pandas as pd
import logging

# Configurar el logging para mostrar advertencias
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Cargar el archivo orders.csv con el delimitador adecuado
orders = pd.read_csv('../data/orders.csv', delimiter=';')

# Verificar y manejar valores faltantes en 'company_id' y 'crate_type'
if orders['company_id'].isnull().any():
    logging.warning("Some rows have missing 'company_id'. These rows will be excluded from the distribution calculation.")
    orders = orders.dropna(subset=['company_id'])  # Excluir filas sin 'company_id'

if orders['crate_type'].isnull().any():
    logging.warning("Some rows have missing 'crate_type'. These rows will be excluded from the distribution calculation.")
    orders = orders.dropna(subset=['crate_type'])  # Excluir filas sin 'crate_type'

# Verificar tipos de datos en 'company_id' y 'crate_type'
if not pd.api.types.is_string_dtype(orders['company_id']):
    logging.warning("'company_id' is not of string type. Converting to string.")
    orders['company_id'] = orders['company_id'].astype(str)

if not pd.api.types.is_string_dtype(orders['crate_type']):
    logging.warning("'crate_type' is not of string type. Converting to string.")
    orders['crate_type'] = orders['crate_type'].astype(str)

# Lista de valores permitidos en 'crate_type'
valid_crate_types = ["Plastic", "Wood", "Metal"]

# Verificar si 'crate_type' contiene valores fuera de los permitidos
invalid_crate_types = orders[~orders['crate_type'].isin(valid_crate_types)]
if not invalid_crate_types.empty:
    logging.warning(f"Some rows contain invalid crate types. These will be excluded from the calculation.\n{invalid_crate_types['crate_type'].unique()}")
    orders = orders[orders['crate_type'].isin(valid_crate_types)]

# Comprobación de duplicados en 'order_id'
if orders['order_id'].duplicated().any():
    logging.warning("Some rows contain duplicate 'order_id'. These rows may indicate duplicate orders.")

# Calcular la distribución de tipos de cajas por empresa después de las verificaciones
distribution = orders.groupby(['company_id', 'crate_type']).size().reset_index(name='order_count')

# Mostrar los primeros resultados para verificar el cálculo
distribution.head()

,company_id,crate_type,order_count
0,012f20c6-00d5-4f45-999f-12e7639db623,Plastic,1
1,063a7dc7-b93a-4f38-b7f0-0e30b5b217ac,Wood,1
2,0b8755d4-3d28-4039-b9a7-b30cb5ff02ea,Wood,1
3,0d09ae2b-d9a5-4d67-bb97-963be9379b4e,Plastic,1
4,0f05a8f1-2bdf-4be7-8c82-4c9b58f04898,Plastic,1


## task 2




In [14]:
import json

def get_full_name(contact_data):
    """Extracts full name from contact data or returns 'John Doe' if incomplete."""
    try:
        # Load the contact data as a JSON object
        contact_info = json.loads(contact_data)[0]  # Access the dictionary inside the list
        first_name = contact_info.get("contact_name", "").strip()
        last_name = contact_info.get("contact_surname", "").strip()
        
        # If either first or last name is missing, default to "John Doe"
        if not first_name or not last_name:
            return "John Doe"
        return f"{first_name} {last_name}"
    except (json.JSONDecodeError, IndexError, TypeError, KeyError):
        # In case of unexpected structure, default to "John Doe"
        return "John Doe"

In [15]:
import pandas as pd

# Load data
file_path = '../data/orders.csv'
orders_df = pd.read_csv(file_path, delimiter=';')

# Function to extract the full name
import json

def get_full_name(contact_data):
    """Extracts full name from contact data or returns 'John Doe' if incomplete."""
    try:
        # Load the contact data as a JSON object
        contact_info = json.loads(contact_data)[0]  # Access the dictionary inside the list
        first_name = contact_info.get("contact_name", "").strip()
        last_name = contact_info.get("contact_surname", "").strip()
        
        # If either first or last name is missing, default to "John Doe"
        if not first_name or not last_name:
            return "John Doe"
        return f"{first_name} {last_name}"
    except (json.JSONDecodeError, IndexError, TypeError, KeyError):
        # In case of unexpected structure, default to "John Doe"
        return "John Doe"

# Apply the function to create contact_full_name
orders_df['contact_full_name'] = orders_df['contact_data'].apply(get_full_name)

# Select relevant columns
contact_fullname_df = orders_df[['order_id', 'contact_full_name']]

# Display the resulting DataFrame
contact_fullname_df.head()

,order_id,contact_full_name
0,f47ac10b-58cc-4372-a567-0e02b2c3d479,Curtis Jackson
1,f47ac10b-58cc-4372-a567-0e02b2c3d480,Maria Theresa
2,f47ac10b-58cc-4372-a567-0e02b2c3d481,Para Cetamol
3,f47ac10b-58cc-4372-a567-0e02b2c3d482,John Doe
4,f47ac10b-58cc-4372-a567-0e02b2c3d483,John Doe
